# Import Libraries

In [21]:
import pandas as pd 
import numpy as np  
import sidetable
import sklearn
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import kaggle
from pathlib import Path
from glob import glob

In [10]:
kaggle.api.authenticate()

In [12]:
path = Path.cwd().parent
path

WindowsPath('e:/Bank-Telemarketing')

# Display Settings

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
import warnings
warnings.filterwarnings('ignore')

In [38]:
path = Path.cwd().parent
path_data = path/'data'/'raw'
file = []
for i in path_data.glob('*.txt'):
    file.append(i)

file

[WindowsPath('e:/Bank-Telemarketing/data/raw/bank-additional-names.txt'),
 WindowsPath('e:/Bank-Telemarketing/data/raw/info.txt')]

In [81]:
f = open(path_data/'bank-additional-names.txt','rt')
for line in f.readlines():
    print(line.strip())

Citation Request:
This dataset is publicly available for research. The details are described in [Moro et al., 2014].
Please include this citation if you plan to use this database:

[Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, In press, http://dx.doi.org/10.1016/j.dss.2014.03.001

Available at: [pdf] http://dx.doi.org/10.1016/j.dss.2014.03.001
[bib] http://www3.dsi.uminho.pt/pcortez/bib/2014-dss.txt

1. Title: Bank Marketing (with social/economic context)

2. Sources
Created by: Sérgio Moro (ISCTE-IUL), Paulo Cortez (Univ. Minho) and Paulo Rita (ISCTE-IUL) @ 2014

3. Past Usage:

The full dataset (bank-additional-full.csv) was described and analyzed in:

S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems (2014), doi:10.1016/j.dss.2014.03.001.

4. Relevant Information:

This dataset is based on "Bank Marketing" UC

In [71]:
df = pd.read_fwf(path_data/'bank-additional-names.txt')#,on_bad_lines='warn',lineterminator='\n')
df

,Citation Request:,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,This dataset is publicly available for researc...,NaN,NaN,NaN,NaN,NaN,NaN
1,Please include this citation if you plan to us...,NaN,NaN,NaN,NaN,NaN,NaN
2,"[Moro et al., 2014] S. Moro, P. Cortez and P. ...",NaN,NaN,NaN,NaN,NaN,NaN
3,Available at: [pdf] http://dx.doi.org/10.1016/...,NaN,NaN,NaN,NaN,NaN,NaN
4,[bib] http://www3.dsi.uminho.pt/pcortez/bib/20...,NaN,NaN,NaN,NaN,NaN,NaN
5,1. Title: Bank Marketing (with social/economic...,NaN,NaN,NaN,NaN,NaN,NaN
6,2. Sources,NaN,NaN,NaN,NaN,NaN,NaN
7,"Created by: Sérgio Moro (ISCTE-IUL), Paulo Cor...",NaN,NaN,NaN,NaN,NaN,NaN
8,3. Past Usage:,NaN,NaN,NaN,NaN,NaN,NaN
9,The full dataset (bank-additional-full.csv) wa...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = pd.read_csv(path/'data'/'raw'/'bank-additional-full.csv', sep=';')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no


In [5]:
df.tail()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.10,94.77,-50.80,1.03,4963.60,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.10,94.77,-50.80,1.03,4963.60,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.10,94.77,-50.80,1.03,4963.60,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.10,94.77,-50.80,1.03,4963.60,yes
41187,74,retired,married,professional.course,no,yes,no,cellular,nov,fri,239,3,999,1,failure,-1.10,94.77,-50.80,1.03,4963.60,no


In [6]:
df.sample()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
39605,52,admin.,married,university.degree,no,yes,yes,cellular,may,thu,250,1,3,5,success,-1.80,93.88,-40.00,0.68,5008.70,yes


In [7]:
df.describe(include='all')

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
count,41188.00,41188,41188,41188,41188,41188,41188,41188,41188,41188,41188.00,41188.00,41188.00,41188.00,41188,41188.00,41188.00,41188.00,41188.00,41188.00,41188
unique,NaN,12,4,8,3,3,3,2,10,5,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,2
top,NaN,admin.,married,university.degree,no,yes,no,cellular,may,thu,NaN,NaN,NaN,NaN,nonexistent,NaN,NaN,NaN,NaN,NaN,no
freq,NaN,10422,24928,12168,32588,21576,33950,26144,13769,8623,NaN,NaN,NaN,NaN,35563,NaN,NaN,NaN,NaN,NaN,36548
mean,40.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,258.29,2.57,962.48,0.17,NaN,0.08,93.58,-40.50,3.62,5167.04,NaN
std,10.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259.28,2.77,186.91,0.49,NaN,1.57,0.58,4.63,1.73,72.25,NaN
min,17.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,1.00,0.00,0.00,NaN,-3.40,92.20,-50.80,0.63,4963.60,NaN
25%,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.00,1.00,999.00,0.00,NaN,-1.80,93.08,-42.70,1.34,5099.10,NaN
50%,38.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.00,2.00,999.00,0.00,NaN,1.10,93.75,-41.80,4.86,5191.00,NaN
75%,47.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,319.00,3.00,999.00,0.00,NaN,1.40,93.99,-36.40,4.96,5228.10,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

# Formatting and Consistency Checks

In [9]:
pd.DataFrame({'dtype':df.dtypes,
              'uique': [df[col].unique() for col in df.columns]})

,dtype,uique
age,int64,"[56, 57, 37, 40, 45, 59, 41, 24, 25, 29, 35, 5..."
job,object,"[housemaid, services, admin., blue-collar, tec..."
marital,object,"[married, single, divorced, unknown]"
education,object,"[basic.4y, high.school, basic.6y, basic.9y, pr..."
default,object,"[no, unknown, yes]"
housing,object,"[no, yes, unknown]"
loan,object,"[no, yes, unknown]"
contact,object,"[telephone, cellular]"
month,object,"[may, jun, jul, aug, oct, nov, dec, mar, apr, ..."
day_of_week,object,"[mon, tue, wed, thu, fri]"
